In [1]:
config = {
    'time': '01:00',
    
    'auto_size': False,
    'W': 660, # ignored when auto_size = True
    'H': 200, # ignored when auto_size = True
    
    'auto_center': True,
    'x': 100, # ignored when auto_center = True
    'y': 10, # ignored when auto_center = True
    
    'font_size': 200,
    # 'font_face': '/Library/Fonts/Arial Rounded Bold.ttf',
    'font_face': '/Users/paulo/Library/Fonts/From Cartoon Blocks.ttf',
    'text_color': '#000000',
    'bg_color': '#ffffff',
    
    'fps': 10,
    'make_gif': 0,
    'make_zero': 1,
    'make_mp4': 1
}

In [2]:
# REFERENCE
# https://stackoverflow.com/questions/43060479/how-to-get-the-font-pixel-height-using-pil-imagefont
# https://pillow.readthedocs.io/en/3.1.x/reference/ImageFont.html

In [3]:
import string
from PIL import Image, ImageDraw, ImageFont
import re
import datetime, time
import os
import cv2

In [13]:
def generate_countdown(config):

SyntaxError: unexpected EOF while parsing (<ipython-input-13-77d3018e11f0>, line 1)

In [4]:
time_multipliers = [1, 60, 3600] # time parts will be reversed too to acomodate all lenghts of time.

time_string = config['time']
font_size = int(config['font_size'])
W = int(config['W'])
H = int(config['H'])
text_color = config['text_color']
bg_color = config['bg_color']
font_face = config['font_face']
font_face_for_filepath = re.compile('[^0-9a-z]+').sub('', os.path.basename(font_face).lower())
fps = int(config['fps'])
make_gif = 1 and int(config['make_gif'])
make_zero = 1 and int(config['make_zero'])
make_mp4 = 1 and int(config['make_mp4'])
auto_center = 1 and int(config['auto_center'])
auto_size = 1 and int(config['auto_size'])
x = int(config['x'])
y = int(config['y'])

In [5]:
# PARAMETERS
time_parts_factors = list(zip(reversed(time_string.split(':')), time_multipliers))
time_parts_multiplied = [ int('0' + a.replace(' ','')) * b for a,b in time_parts_factors ]
seconds = sum(time_parts_multiplied)

if seconds > 10 * time_multipliers[2]: # 24 hours
    seconds = 10
    raise ValueError('Time can\'t be higher than 10:00:00')

In [6]:
# OBJECTS and ADDITIONAL METRICS
slice_position = 0
if seconds < 60 * 60:
    slice_position = 2
initial_text = str(datetime.timedelta(seconds=seconds))[slice_position:] # necessary for base calculations.
fnt = ImageFont.truetype(font_face, font_size)
box = fnt.getmask(initial_text).getbbox()
if auto_size:
    W, H = int(box[2] * 1.1), int(box[3] * 1.4)
img = Image.new('RGBA', (W,H), color = bg_color)
draw = ImageDraw.Draw(img)
(width, baseline), (offset_x, offset_y) = fnt.font.getsize(initial_text)
w, h = draw.textsize(initial_text, font=fnt)

In [7]:
# GENERATE FOLDER AND FILES
style_folder = '_'.join(map(str, [
    W,
    H,
    font_size,
    text_color.replace('#',''),
    bg_color.replace('#',''),
    font_face_for_filepath
]))
if not os.path.isdir(style_folder):
    os.mkdir(style_folder)

In [8]:
frames = []
for t in range(seconds, -1, -1):
    text = str(datetime.timedelta(seconds=t))[slice_position:]
    img = Image.new('RGB', (W,H), color=bg_color)
    draw = ImageDraw.Draw(img)
    if auto_center:
        x = (W-w-offset_x)/2
        y = (H-h-offset_y)/2
    draw.text((x,y), text, fill=text_color, font=fnt)
    frames.append(img)

In [9]:
if make_gif:
    gif_filename = os.path.join(style_folder, str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '.gif')
    frames[0].save(
        gif_filename,
        format='GIF',
        append_images=frames[1:],
        save_all=True,
        duration=int(1000/fps),
        loop=0,
        transparency=0,
        disposal=2
    )

In [10]:
if make_zero:
    zero_filename = 'zero.png'
    frames[len(frames) - 1].save(os.path.join(style_folder, zero_filename))
    first_frame_filename = str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '-static.png'
    frames[0].save(os.path.join(style_folder, first_frame_filename))

In [11]:
if make_mp4:
    video_filename = os.path.join(style_folder, str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '.mp4')
    import numpy as np
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video = cv2.VideoWriter(video_filename,fourcc, fps, frames[0].size)
    for frame in frames:
        imtemp = frame.copy()
        video.write(np.array(imtemp))
    video.release()

In [12]:
for frame in frames:
    frame.close()